1. Setup and Initial Data Loading

    Environment Setup: Ensure all necessary Python libraries are installed (geopandas, rasterio, rioxarray, numpy, skfuzzy, etc.).
    Load Grassland Boundary Data: Use geopandas to load the grassland boundary data from a shapefile or other geospatial file format.

2. Data Acquisition and Preprocessing

    Identify and Acquire Datasets: Based on the grassland boundaries, identify the relevant datasets (soil, elevation, climate) you need.
    Data Preprocessing:
        Spatial Alignment: Align all raster datasets to the same spatial extent and resolution, ensuring they match the grassland boundary.
        Data Cleaning: Handle any missing or anomalous data in your datasets.

3. Fuzzy Logic Model Setup

    Define Fuzzy Variables: Create fuzzy variables (Antecedents and Consequents) for each environmental factor (e.g., soil pH, elevation) and the output (e.g., habitat suitability).
    Define Membership Functions: Assign membership functions (e.g., triangular, trapezoidal) to each fuzzy set within your fuzzy variables.

4. Rule Definition

    Create Fuzzy Rules: Define the fuzzy logic rules that relate your input variables (environmental factors) to your output variable (habitat suitability).

5. Model Integration and Execution

    Integrate Spatial Data with Fuzzy Model:
        For each pixel or spatial unit in your aligned datasets, extract the necessary values (e.g., soil pH, elevation).
        Input these values into your fuzzy logic model to compute habitat suitability.
    Run the Model: Use the fuzzy logic system to process the data and output habitat suitability scores.

6. Post-processing and Visualization

    Generate Output Raster: Convert the habitat suitability scores back into a raster format, aligning it with your original spatial data.
    Visualization: Use tools like matplotlib to visualize the habitat suitability across the grassland area.

7. Analysis and Interpretation

    Interpret Results: Analyze the habitat suitability map to draw conclusions about the most suitable areas for Sorghastrum nutans.
    Validation (Optional): If validation data is available, compare your model's output to assess its accuracy.

8. Documentation and Reporting

    Document the Workflow: Ensure that each step of your process is well-documented.
    Prepare a Final Report: Summarize your methodology, findings, and any conclusions or recommendations.

Usage of Grassland Boundary

    Initial Data Filtering: Use the grassland boundary to filter or clip your environmental datasets right after loading them. This ensures that all subsequent analyses focus only on the relevant areas.
    Spatial Analysis: In the model integration step, use the boundary to ensure that habitat suitability is only calculated for areas within the grassland.

This structure provides a comprehensive approach to building and executing your fuzzy logic model, ensuring that spatial considerations are integrated effectively with fuzzy logic principles.

In [9]:
# Set Envionmentals

# Imports
import os
from glob import glob
import pathlib
import json
#Third Party
import earthpy as et
import earthpy.appeears as etapp
# import earthpy.earthexplorer as 
import geopandas as gpd
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
# Inlcude rest of modules as needed
with open('settings.json', 'r') as file:
    config = json.load(file)

# # Set data directory globals here
# THIS CAN BE DONE THROUGH A JSON ONLY NEED TO LOOP
# THROUGH TO  CHECK IF FILES EXIST

# data_dir = ('/workspaces/golden-feather-grass-habitats/Assets/')
# grassland_path = os.path.join(data_dir, 'study-site-boundaries')
# raster_bin = os.path.join(data_dir, 'bin', 'rasters')
# soil_dir = os.path.join(raster_bin, 'soil')
# climate_dir = os.path.join(raster_bin, 'climate')
# dem_dir = os.path.join(raster_bin, 'DEM')
# topo_dir = os.path.join(raster_bin, 'topo-derive')
# vectors_dir = os.path.join(data_dir, 'vector')

# directories = [grassland_path, 
#                raster_bin, 
#                soil_dir, 
#                climate_dir, 
#                dem_dir, 
#                topo_dir, 
#                vectors_dir]

# for a_dir in (directories):
#     if not os.path.exists(a_dir):
#         os.makedirs(a_dir)

In [15]:
# Get boundary gdf for clipping

# Download and load boundary data
boundary_config = config['boundary']
boundary_gdf = gpd.read_file(boundary_config['url'])
boundary_gdf = boundary_gdf.to_crs(boundary_config['crs'])

boundary_gdf


# study_site-gdf = grass_gdf[['SITES HERE']]

# Make sure to check crs 

# Can run through study site gdf as loop later

#Plot code here to test geometry

HTTPError: HTTP Error 404: Not Found

In [ ]:
#Define import functions here
# Check if arrays or downloads already exist for each layer,  if not 
# download and create array
# Merge arrays if necessary 

    
# soils 
def soil_downloader():
    
# elevation <-- DEM from appears
def dem_downloader(x1):
# climate <--- check elsa's post
# derived topo <-- ask chatgpt + check old project 

In [ ]:
#Prune and Harmonization functions here
# Clip arrays to boundary layer, make new clipped object 
# Check if all arrays have same crs, area, and bounds
# throw error if not, print values to compare
# get rid of unneccesary layers

# Example of opening a raster file and clipping it to the grassland boundary
with rasterio.open('path_to_raster_file') as src:
    raster_data = src.read(1)
    raster_affine = src.transform

# Convert the raster data to an xarray DataArray
raster_xr = rioxarray.open_rasterio('path_to_raster_file')

# Clip the raster data to the grassland boundary
clipped_raster = raster_xr.rio.clip(grassland_boundaries.geometry.apply(mapping), grassland_boundaries.crs)





In [ ]:
# start building fuzzy logic model here 

# Example: Defining fuzzy variables and sets
soil_pH = ctrl.Antecedent(np.arange(0, 14, 1), 'soil_pH')
elevation = ctrl.Antecedent(np.arange(0, 3000, 1), 'elevation')  # Example range
precipitation = ctrl.Antecedent(np.arange(0, 500, 1), 'precipitation')  # Example range
habitat_suitability = ctrl.Consequent(np.arange(0, 100, 1), 'habitat_suitability')

# Define membership functions
soil_pH['acidic'] = fuzz.trimf(soil_pH.universe, [0, 0, 5.5])
# ... continue for other sets

# Define rules
rule1 = ctrl.Rule(soil_pH['neutral'] & elevation['medium'] & precipitation['moderate'], habitat_suitability['high'])
# ... more rules

# Control system
habitat_ctrl = ctrl.ControlSystem([rule1, ...])
habitat_simulation = ctrl.ControlSystemSimulation(habitat_ctrl)

# Applying the model to data
# This would involve iterating over your raster data, applying the model to each pixel
# Example:
habitat_simulation.input['soil_pH'] = 6.5  # Example input
# ... set other inputs
habitat_simulation.compute()
print(habitat_simulation.output['habitat_suitability'])

# Visualization code goes here